In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.3 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 50.6 MB/s eta 0:00:01
     |████████████████████████████████| 981 kB 30.3 MB/s eta 0:00:01
     |████████████████████████████████| 263 kB 48.9 MB/s eta 0:00:01
     |████████████████████████████████| 468 kB 55.5 MB/s eta 0:00:01


  Created wheel for ktrain: filename=ktrain-0.26.1-py3-none-any.whl size=25277415 sha256=ff46badeef5a2ac11b79981d0b53be76ffa87b747e194e7dae4713e8ac01478e
  Stored in directory: /root/.cache/pip/wheels/2d/ab/6f/c96fd66c323253adda904c2badd8abf98def41906ed29c3a21
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9918 sha256=44a91a677c1fd99beeb7b86df6756e9683682ef54216a6a474172876d53b5017
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff05c722d142b7d20a88ad63f9aab11b895411241a4
  Created wheel for keras-bert: filename=keras_bert-0.86.0-py3-none-any.whl size=34143 sha256=fcf0c5b7fa73741d13cbe7054aadd7107b208b59dc04e34d2e27ba00fd380b34
  Stored in directory: /root/.cache/pip/wheels/fc/c1/0a/eb9187261b3f192ac314aefb54fe66f50540c3edb906599633
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-py3-none-any.whl size=12941 sha256=acca60b8e10bd7e5b4fb55152862b540b9dd2de104231ed23efa7a3c530af51f
  Stored in directory: /root/.cache/pip/w

In [2]:
!wget https://raw.githubusercontent.com/ComputerGuyYash/DeepN_snippets/main/predictor.py -O /opt/conda/lib/python3.7/site-packages/ktrain/text/predictor.py

--2021-03-13 17:34:08--  https://raw.githubusercontent.com/ComputerGuyYash/DeepN_snippets/main/predictor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11421 (11K) [text/plain]
Saving to: ‘/opt/conda/lib/python3.7/site-packages/ktrain/text/predictor.py’

/opt/conda/lib/pyth 100%[===================>]  11.15K  --.-KB/s    in 0.001s  

2021-03-13 17:34:08 (15.9 MB/s) - ‘/opt/conda/lib/python3.7/site-packages/ktrain/text/predictor.py’ saved [11421/11421]



In [3]:
import pandas as pd
dataset = pd.read_csv("../input/deepnrefined/td.csv")

In [4]:
# dataset

In [5]:
dataset = dataset[:-2]

In [6]:
x = dataset.Text.values
y = dataset.rumour_identification.values.astype(int)
y[y==-9223372036854775808]=0


In [7]:
x0 = (x[y==0]).tolist()

In [8]:
x1 = (x[y==1][:1367]).tolist()

In [9]:
y = [0 for x in range(1367)]
y = y+[1 for x in range(1367)]

In [10]:
x = x0+x1

In [11]:
x_train = x[:-100]
y_train = y[:-100]
x_test = x[-100:]
y_test = y[-100:]

In [12]:
set(y_train)

{0, 1}

In [41]:
MODEL_NAME = 'distilbert-base-uncased'

In [46]:
import ktrain
from ktrain import text
t = text.Transformer(MODEL_NAME,maxlen=310,classes=["NO","YES"])

In [15]:
# y_train

In [47]:
trn = t.preprocess_train(x_train,y_train)
val = t.preprocess_test(x_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 27
	95percentile : 35
	99percentile : 37


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 28
	95percentile : 35
	99percentile : 37


In [48]:
model = t.get_classifier()
learner = ktrain.get_learner(model,train_data=trn,val_data=val,batch_size=6)

In [18]:
# learner.lr_find(show_plot=True,max_epochs=2)

In [74]:
learner.fit_onecycle(2e-5,2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
439/439 [==============================] - 61s 138ms/step - loss: 0.0507 - accuracy: 0.9871 - val_loss: 0.7150 - val_accuracy: 0.8000
Epoch 2/2
 14/439 [..............................] - ETA: 57s - loss: 0.0644 - accuracy: 0.9881

KeyboardInterrupt: 

In [ ]:
# learner.fit_onecycle(1e-6,2)

In [75]:
predictor = ktrain.get_predictor(learner.model,preproc=t)

In [68]:
dt = pd.read_csv("../input/deepnrefined/td2.csv")

In [69]:
identity = dt.Identity.values.astype(int)
text = dt.Text.values

In [76]:
ans = []

In [54]:
# i

In [77]:
i = 0
for x,y in zip(identity,text):
    app = []
    app.append(x)
    out = predictor.predict(y)
#     break
    if(out=="NO"):
        app.append(0)
    else:
        app.append(1)
    ans.append(app)
    i+=1
    if(i%100==99):
        print(i)

99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699


In [ ]:
len(ans)

In [78]:
submit = pd.DataFrame(ans,columns=["Identity","rumour_identification"])

In [79]:
submit.to_csv("/kaggle/working/submission.csv",index=False)